In [1]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession
  .builder
  .appName("SummarizerExample")
  .getOrCreate()

import spark.implicits._

spark = org.apache.spark.sql.SparkSession@7ab6f54


org.apache.spark.sql.SparkSession@7ab6f54

⊕ [spark 统计每天新增用户数 - 董可伦 - CSDN博客](https://blog.csdn.net/dkl12/article/details/80256688)
- 对原始数据进行倒排索引
- 这样我们只看列一，统计每个日期在列一出现的次数，即为对应日期新增用户数。

In [2]:
val rdd1 = spark.sparkContext.parallelize(
  Array(
    ("2017-01-01", "a"), ("2017-01-01", "b"), ("2017-01-01", "c"),
    ("2017-01-02", "a"), ("2017-01-02", "b"), ("2017-01-02", "d"),
    ("2017-01-03", "b"), ("2017-01-03", "e"), ("2017-01-03", "f")))
//倒排
val rdd2 = rdd1.map(kv => (kv._2, kv._1))
//倒排后的key分组
val rdd3 = rdd2.groupByKey()
//取最小时间
val rdd4 = rdd3.map(kv => (kv._2.min, 1))
rdd4.countByKey().foreach(println)

(2017-01-01,3)
(2017-01-02,1)
(2017-01-03,2)


rdd1 = ParallelCollectionRDD[0] at parallelize at <console>:31
rdd2 = MapPartitionsRDD[1] at map at <console>:37
rdd3 = ShuffledRDD[2] at groupByKey at <console>:39
rdd4 = MapPartitionsRDD[3] at map at <console>:41


MapPartitionsRDD[3] at map at <console>:41

⊕ [Spark 解决 某商品日交易额统计 - 一路风景 - CSDN博客](https://blog.csdn.net/zrc199021/article/details/73482528)


In [3]:
//加载数据集
val rdd1 = sc.textFile("../../data/sales.txt")

//过滤
val rdd2 = rdd1.filter (log => log.split(",").length == 3)

rdd1 = ../../data/sales.txt MapPartitionsRDD[7] at textFile at <console>:36
rdd2 = MapPartitionsRDD[8] at filter at <console>:39


MapPartitionsRDD[8] at filter at <console>:39

In [17]:
//切分字段
//导入 Row
import org.apache.spark.sql.Row
//取前两个字段
val rowRdd3 = rdd2.map { log => Row(log.split(",")(0), log.split(",")(1).toDouble) }

//导入
import org.apache.spark.sql.types._

//创建映射对应关系
val schema = StructType(Array(
StructField("date", StringType, true),
StructField("sale_amount", DoubleType, true))
)

// 创建df
val saleDF = spark.createDataFrame(rowRdd3, schema)  
import spark.implicits._
import org.apache.spark.sql.functions.sum

saleDF.groupBy("date").agg("date", sum("sale_amount"))
    .rdd.map { row => Row(row(1), row(2)) }
    .coalesce(1,true)
    // .saveAsTextFile("/home/shiyanlou/res")
    .foreach(println)

Name: Unknown Error
Message: <console>:55: error: overloaded method value agg with alternatives:
  (expr: org.apache.spark.sql.Column,exprs: org.apache.spark.sql.Column*)org.apache.spark.sql.DataFrame <and>
  (aggExpr: (String, String),aggExprs: (String, String)*)org.apache.spark.sql.DataFrame
 cannot be applied to (String, org.apache.spark.sql.Column)
       saleDF.groupBy("date").agg("date", sum("sale_amount"))
                              ^

StackTrace: 

⊕ [dataframe - Spark SQL: apply aggregate functions to a list of column - Stack Overflow](https://stackoverflow.com/questions/33882894/spark-sql-apply-aggregate-functions-to-a-list-of-column)


In [8]:
val df = sc.parallelize(Seq(
  (1.0, 0.3, 1.0), (1.0, 0.5, 0.0),
  (-1.0, 0.6, 0.5), (-1.0, 5.6, 0.2))
).toDF("col1", "col2", "col3")

df.groupBy($"col1").min().show
df.groupBy($"col1").sum().show

+----+---------+---------+---------+
|col1|min(col1)|min(col2)|min(col3)|
+----+---------+---------+---------+
|-1.0|     -1.0|      0.6|      0.2|
| 1.0|      1.0|      0.3|      0.0|
+----+---------+---------+---------+

+----+---------+-----------------+---------+
|col1|sum(col1)|        sum(col2)|sum(col3)|
+----+---------+-----------------+---------+
|-1.0|     -2.0|6.199999999999999|      0.7|
| 1.0|      2.0|              0.8|      1.0|
+----+---------+-----------------+---------+



df = [col1: double, col2: double ... 1 more field]


[col1: double, col2: double ... 1 more field]

In [10]:
val df = sc.parallelize(Seq(
  ("tom", 0.3, 1.0), ("tom", 0.5, 0.0),
  ("tim", 0.6, 0.5), ("tim", 5.6, 0.2))
).toDF("col1", "col2", "col3")

df.groupBy($"col1").min().show
df.groupBy($"col1").sum().show

+----+---------+---------+
|col1|min(col2)|min(col3)|
+----+---------+---------+
| tim|      0.6|      0.2|
| tom|      0.3|      0.0|
+----+---------+---------+

+----+-----------------+---------+
|col1|        sum(col2)|sum(col3)|
+----+-----------------+---------+
| tim|6.199999999999999|      0.7|
| tom|              0.8|      1.0|
+----+-----------------+---------+



df = [col1: string, col2: double ... 1 more field]


[col1: string, col2: double ... 1 more field]